In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
import amstools

In [4]:
from amstools.lammps import write_in_lammps

In [6]:
from amstools.qatoms import QAtoms

In [7]:
structure = QAtoms.bulk("Al",cubic=True)*(5,5,5)

In [9]:
structure=structure.sample(frac=0.5, random_state=42).set(element='Li')

In [10]:
structure

QAtoms(symbols='Al250Li250', pbc=True, cell=[20.25, 20.25, 20.25], _selected_mask=..., 250 selected atoms)

# Setup LAMMPS with distilled model

In [19]:
import os

In [21]:
cwd=os.getcwd()

In [22]:
potname=f"{cwd}/../3b-distillation/seed/1/saved_model.yaml"
potname=os.path.abspath(potname)
potname

'/pc2/users/u/usrtr200/mlpfits-2026-grace/grace-tutorial-2026/3-foundation-models/3c-lammps-distilled/../3b-distillation/seed/1/saved_model.yaml'

In [25]:
write_in_lammps(structure=structure, potname=potname, 
                working_dir="lammps-grace-fs-dist",
                with_extrapolation_grade=True,
                steps=[
                     {
                        "type": "min_full",
                        # "name": "my_full_relaxation",  # Optional: custom name for the step
                        "thermo_freq": 100,           # Optional: thermodynamic data output frequency
                        "steps": 1000,                # Optional: number of minimization steps
                        "aniso_style": "aniso",       # Optional: 'iso', 'aniso', or 'tri'
                    },

                    {
                        "type": "mdmc",
                        # "name": "my_mdmc_swap",
                        "thermo_freq": 500,
                        "steps": 50000,
                        "T_MDMC": 300,               # Temperature for Monte Carlo swap attempts
                        "T1": 300, "T2": 300,        # Temperature for MD (NPT)
                        "P1": 0, "P2": 0,              # Pressure for MD (NPT)
                        "aniso_style": "aniso",
                        # "elements": ["Ni", "Co", "Cr"], # Elements to be swapped, by default - all possible pairs
                        "N": 100,                     # Perform M swap attempts every N steps
                        "M": 100,                     # Number of swap attempts to perform
                    },
                    
                ],
                pairstyle='grace/fs extrapolation',
               )
            

# Setup LAMMPS with reference finetuned GRACE-2L

In [26]:
grace_potname=f"{cwd}/../3a-finetuning/seed/1/final_model/"
grace_potname=os.path.abspath(grace_potname)
grace_potname

'/pc2/users/u/usrtr200/mlpfits-2026-grace/grace-tutorial-2026/3-foundation-models/3a-finetuning/seed/1/final_model'

In [27]:
write_in_lammps(structure=structure, potname=grace_potname, 
                working_dir="lammps-grace-2L",
                # with_extrapolation_grade=True,
                steps=[
                     {
                        "type": "min_full",
                        # "name": "my_full_relaxation",  # Optional: custom name for the step
                        "thermo_freq": 100,           # Optional: thermodynamic data output frequency
                        "steps": 1000,                # Optional: number of minimization steps
                        "aniso_style": "aniso",       # Optional: 'iso', 'aniso', or 'tri'
                    },

                    {
                        "type": "mdmc",
                        # "name": "my_mdmc_swap",
                        "thermo_freq": 500,
                        "steps": 50000,
                        "T_MDMC": 300,               # Temperature for Monte Carlo swap attempts
                        "T1": 300, "T2": 300,        # Temperature for MD (NPT)
                        "P1": 0, "P2": 0,              # Pressure for MD (NPT)
                        "aniso_style": "aniso",
                        # "elements": ["Ni", "Co", "Cr"], # Elements to be swapped, by default - all possible pairs
                        "N": 100,                     # Perform M swap attempts every N steps
                        "M": 100,                     # Number of swap attempts to perform
                    },
                    
                ],
                pairstyle='grace',
               )
            